## UPLOAD CSV

In [ ]:
# to upload a csv
from google.colab import files
uploaded = files.upload()

Saving Audiobooks_data.csv to Audiobooks_data.csv


## PREPROCESSING THE DATA

In [ ]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')
raw_csv_data

array([[9.9400e+02, 1.6200e+03, 1.6200e+03, ..., 5.0000e+00, 9.2000e+01,
        0.0000e+00],
       [1.1430e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0590e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 3.8800e+02,
        0.0000e+00],
       ...,
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00],
       [2.5100e+02, 1.6740e+03, 3.3480e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

### Shuffling and balancing the dataset

In [ ]:
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

shuffle_index = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffle_index)

unscaled_inputs_all = unscaled_inputs_all[shuffle_index]
targets_all = targets_all[shuffle_index]

num_of_one_in_target = int(targets_all.sum())
zero_counter =0
indices_to_remove =[]

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_counter += 1
    if zero_counter > num_of_one_in_target:
      indices_to_remove.append(i)

unscaled_balanced_inputs = np.delete(unscaled_inputs_all,indices_to_remove, axis=0)
balanced_targets = np.delete(targets_all,indices_to_remove, axis=0)

### Scaling and splitting and.. shuffle?

In [ ]:
scaled_inputs = preprocessing.scale(unscaled_balanced_inputs)

# shuffling a 2nd time is essential because the targets with 0 s are selected orderly from the beginning, 
# hence it was found that validation and test datasets had only 1 s as the targets
shuffle1_index = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffle1_index)

scaled_inputs = scaled_inputs[shuffle1_index]
balanced_targets = balanced_targets[shuffle1_index]



# using a 80 : 10 : 10 spilt of the data
input_count = scaled_inputs.shape[0]
train_count = int(0.8*input_count)
validation_count = int(0.1*input_count)
test_count = input_count -train_count -validation_count

train_inputs = scaled_inputs[:train_count]
validation_inputs = scaled_inputs[train_count: validation_count+train_count]
test_inputs = scaled_inputs[train_count + validation_count: input_count]

train_targets = balanced_targets[:train_count]
validation_targets = balanced_targets[train_count: validation_count+train_count]
test_targets = balanced_targets[train_count + validation_count: input_count]

In [ ]:
# checking if the processed inputs are balanced
print(np.sum(train_targets)/train_count,'\n',np.sum(validation_targets)/validation_count, '\n',np.sum(test_targets)/test_count)

0.4984632578932663 
 0.5055928411633109 
 0.5066964285714286


### saving as .npz

In [ ]:
np.savez('audiobooks_data_train', inputs = train_inputs, targets = train_targets)
np.savez('audiobooks_data_validation', inputs = validation_inputs, targets = validation_targets)
np.savez('audiobooks_data_test', inputs = test_inputs, targets = test_targets)


### Loading npz files

In [ ]:
npz = np.load('audiobooks_data_train.npz')

train_in = npz['inputs'].astype(np.float64)
train_tar = npz['targets'].astype(np.int32)

npz = np.load('audiobooks_data_validation.npz')
validation_in , validation_tar = npz['inputs'].astype(np.float64) , npz['targets'].astype(np.int32)

npz = np.load('audiobooks_data_test.npz')
test_in , test_tar = npz['inputs'].astype(np.float64) , npz['targets'].astype(np.int32)

## MODEL

An important note on activation functions:
1. Since Relu makes the nodes having negative values deactivated, it makes many of these less useful. But this is precisely what we wanted for MNIST classification , for quicker predictions
2. Softmax is generally used only for the layer preceeding the output layer, as probabilistic output of its activation may not be desired in between


In [ ]:
import tensorflow as tf

input_size = 10
output_size = 2
hidden_layer_size =50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer = 'adam', loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])



In [ ]:
batch_size = 100
max_epochs = 100 

# 'patience' lets us decide how many instances of loss increases we can tolerate
# EarlyStopping(patience)
early_stopping = tf.keras.callbacks.EarlyStopping(patience =2)

# while training with arrays batching can be done within model.fit
model.fit(train_in, train_tar,
          batch_size = batch_size,
          epochs = max_epochs,
          callbacks = [early_stopping],
          validation_data =(validation_in, validation_tar),
          verbose =2
          )


Epoch 1/100
36/36 - 0s - loss: 0.3417 - accuracy: 0.8156 - val_loss: 0.2959 - val_accuracy: 0.8322 - 125ms/epoch - 3ms/step
Epoch 2/100
36/36 - 0s - loss: 0.3418 - accuracy: 0.8145 - val_loss: 0.2861 - val_accuracy: 0.8501 - 86ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3400 - accuracy: 0.8117 - val_loss: 0.2858 - val_accuracy: 0.8367 - 89ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3428 - accuracy: 0.8136 - val_loss: 0.2929 - val_accuracy: 0.8501 - 94ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3396 - accuracy: 0.8175 - val_loss: 0.2972 - val_accuracy: 0.8367 - 88ms/epoch - 2ms/step


In [ ]:
test_loss ,test_accuracy = model.evaluate(test_in, test_tar)

14/14 [==============================] - 0s 2ms/step - loss: 0.3421 - accuracy: 0.8214
